In [9]:
import pandas as pd
import numpy as np 

VAL_RATIO = 0.2 

def prepare_csv(seed=999):
    file_path = '../data/reddit_train.csv'
    file_path2 = '../data/reddit_test.csv'
    df_train = pd.read_csv(file_path)
    df_train["comments"] = df_train.comments.str.replace("\n", " ")
    idx = np.arange(df_train.shape[0])
    np.random.seed(seed)
    np.random.shuffle(idx)
    val_size = int(len(idx) * VAL_RATIO)
    df_train.iloc[idx[val_size:], :].to_csv("../data/dataset_train.csv", index=False)
    df_train.iloc[idx[:val_size], :].to_csv("../data/dataset_val.csv", index=False)
    df_test = pd.read_csv(file_path2)
    df_test["comments"] = df_test.comments.str.replace("\n", " ")
    df_test.to_csv("../data/dataset_test.csv", index=False)

In [13]:
import re 

import spacy
NLP = spacy.load('en')
MAX_CHARS = 20000

def tokenizer(comment):
    comment = re.sub(r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;]", " ", str(comment))
    comment = re.sub(r"[ ]+", " ", comment)
    comment = re.sub(r"\!+", "!", comment)
    comment = re.sub(r"\,+", ",", comment)
    comment = re.sub(r"\?+", "?", comment)
    if (len(comment) > MAX_CHARS):
        comment = comment[:MAX_CHARS]
    return [x.text for x in NLP.tokenizer(comment) if x.text != " "]

In [ ]:
import logging 

import torch
from torchtext import data

LOGGER = logging.getLogger("Reddit_dataset")

def get_datast(fix_lenght=100, lower=False, vectors=None):
    if vectors is not None:
        lower=True
    LOGGER.debug("Preparing CSV files....")
    prepare_csv()
    comment = data.Field(sequential=True, fix_length=fix_length, tokenizer=tokenizer,
                        pad_first=True, tensor_type=torch.cuda.LongTensor, lower=lower)
    LOGGER.debug("Reading train CSV file......")
    train, val = data.TabularDataset.splits(path='../data/', format='csv', skip_header=True
                                           train='dataset_train,csv', validation='dataset_val.csv',
                                           fields=[
                                               ('id', None), 
                                               ('comments', comment),
                                               
                                           ])